# Official

In [2]:
from pyspark.sql import SparkSession
import numpy as np
import os
from pyspark.sql import functions as F
from pyspark.sql.functions import col, concat_ws, rand, sum, when, isnull, count, isnan

In [ ]:
spark = SparkSession.builder \
        .appName("Hackathon") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.driver.maxResultSize", "0") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.5.0") \
        .getOrCreate()

In [4]:
pathing_review = "datasets/review_data/"
arr = np.array(os.listdir(pathing_review))
reviewData_files = pathing_review + arr

pathing_metadata = "datasets/review_metadata/"
arr = np.array(os.listdir(pathing_metadata))
reviewMetadata_files = pathing_metadata + arr

df_review = spark.read.json(list(reviewData_files)).dropna(subset="text").drop_duplicates()
df_metadata = spark.read.json(list(reviewMetadata_files)).dropna(subset="category").drop_duplicates().withColumnRenamed("name", "business_name").select(["gmap_id", "category", "business_name"])

df_joined = df_review.join(df_metadata, on="gmap_id", how="inner").withColumn("category_str", concat_ws(", ", col("category"))).withColumn("random_order", rand()).orderBy("random_order").drop("random_order")

In [5]:
df_metadata.printSchema()

root
 |-- gmap_id: string (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- business_name: string (nullable = true)



In [6]:
df_review.printSchema()

root
 |-- gmap_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- rating: long (nullable = true)
 |-- resp: struct (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- time: long (nullable = true)
 |-- text: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)



In [7]:
df_joined.printSchema()

root
 |-- gmap_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- rating: long (nullable = true)
 |-- resp: struct (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- time: long (nullable = true)
 |-- text: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- business_name: string (nullable = true)
 |-- category_str: string (nullable = false)



In [8]:
from pyspark.sql.functions import col, when, count, size, trim, round
from pyspark.sql.types import StringType, FloatType, DoubleType, ArrayType, MapType

# 1) Drop unwanted columns
df_filtered = df_joined.drop("pics", "resp", "time", "category", "user_id")

# 2) Build a "is missing" condition per column based on its data type
missing_conds = []
for f in df_filtered.schema.fields:
    c = col(f.name)
    dt = f.dataType

    if isinstance(dt, (FloatType, DoubleType)):
        # floats/doubles: NULL or NaN
        cond = c.isNull() | c.isnan()
    elif isinstance(dt, StringType):
        # strings: NULL or empty after trim
        cond = c.isNull() | (trim(c) == "")
    elif isinstance(dt, (ArrayType, MapType)):
        # arrays/maps: NULL or empty
        cond = c.isNull() | (size(c) == 0)
    else:
        # ints/longs/booleans/date/timestamp/structs: only NULL
        cond = c.isNull()

    missing_conds.append(count(when(cond, True)).alias(f.name))


In [9]:
missing_counts = df_filtered.select(missing_conds)
missing_counts.show(truncate=False)

+-------+----+------+----+-------------+------------+
|gmap_id|name|rating|text|business_name|category_str|
+-------+----+------+----+-------------+------------+
|0      |0   |0     |0   |0            |0           |
+-------+----+------+----+-------------+------------+



### Some reviews have translation, we only need English ones

In [10]:
df_joined.filter(df_joined.text.contains("(Translated by Google)")).select("text").show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|                        (Translated by Google) Good fast service\n\n(Original)\nBuen servicio rápido|
|(Translated by Google) They do not want to sell carnitas with lots of fat. I kick the compass.\n\...|
|(Translated by Google) It's worth it even if it's retired.\n\n(Original)\nVale la pena aunque est...|
|(Translated by Google) I was here in the morning here at Rapid Wash Laundry and it is heat is alm...|
|                                      (Translated by Google) Very good...\n\n(Original)\nMuy bien...|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows



### Some reviews have many spacings, remove spacings for one-line reviews

In [11]:
df_joined.filter(F.col("text").rlike("\n")).select("text").show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|Lake filled with fish, actually a little hilly, plenty of room to walk and wander.  Was a nice br...|
|                        (Translated by Google) Good fast service\n\n(Original)\nBuen servicio rápido|
|Nice people, but can't sit down. Long long line at drive through.  Food was cold, and not too goo...|
|(Translated by Google) They do not want to sell carnitas with lots of fat. I kick the compass.\n\...|
|(Translated by Google) It's worth it even if it's retired.\n\n(Original)\nVale la pena aunque est...|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows



In [12]:
# Removing (Translated by Google) prefix and (Original) languages to get the English reviews 
# Removing all newlines for one-lined reviews
# Removing quotation marks

df_joined = df_joined.withColumn(
    "text",
    F.when(
        F.col("text").contains("(Translated by Google)"),
        # extract the English text, remove newlines, remove quotes
        F.regexp_replace(
            F.regexp_replace(
                F.regexp_extract(F.col("text"), r"\(Translated by Google\)\s*([^\n]+)", 1),
                r"\n+", " "
            ),
            r"\"", ""
        )
    ).otherwise(
        # for rows without Google Translate tag, remove newlines and quotes
        F.regexp_replace(
            F.regexp_replace(F.col("text"), r"\n+", " "),
            r"\"", ""
        )
    )
)


In [13]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

spark_nlp = sparknlp.start(apple_silicon=True)

25/08/30 11:32:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# -------------------------------
# 1. Document Assembler
# -------------------------------
customer_review = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("customer_review")

# -------------------------------
# 2. Tokenizer
# -------------------------------
customer_review_token = Tokenizer() \
    .setInputCols(["customer_review"]) \
    .setOutputCol("customer_review_token")

# -------------------------------
# 3. Spell Checker
# -------------------------------
customer_review_spell_checker = NorvigSweetingModel.pretrained() \
    .setInputCols(["customer_review_token"]) \
    .setOutputCol("customer_review_corrected")

# -------------------------------
# 4. Normalizer (lowercasing, clean text)
# -------------------------------
customer_review_normalizer = Normalizer() \
    .setInputCols(["customer_review_corrected"]) \
    .setOutputCol("customer_review_normalized") \
    .setLowercase(True)

# -------------------------------
# 5. StopWords Cleaner
# -------------------------------
customer_review_stopwordsCleaner = StopWordsCleaner() \
    .setInputCols(["customer_review_normalized"]) \
    .setOutputCol("customer_review_cleaned")

# -------------------------------
# 6. Lemmatizer
# -------------------------------
customer_review_lemma = LemmatizerModel.pretrained() \
    .setInputCols(["customer_review_token"]) \
    .setOutputCol("customer_review_lemma")

# -------------------------------
# 7. Word Embeddings (GloVe)
# -------------------------------
glove_embeddings = WordEmbeddingsModel.pretrained("glove_100d") \
    .setInputCols(["customer_review_token", "customer_review"]) \
    .setOutputCol("word_embeddings")

# -------------------------------
# 8. Sentence Embeddings (average pooling)
# -------------------------------
sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["customer_review", "word_embeddings"]) \
    .setOutputCol("customer_review_embeddings") \
    .setPoolingStrategy("AVERAGE")

# -------------------------------
# 9. Embeddings Finisher (convert to Spark vector/array)
# -------------------------------
customer_review_finisher = EmbeddingsFinisher() \
    .setInputCols(["customer_review_embeddings"]) \
    .setOutputCols(["customer_review_vector"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

In [ ]:
# -------------------------------
# 1. Document Assembler
# -------------------------------
business_category = DocumentAssembler() \
    .setInputCol("category_str") \
    .setOutputCol("business_category")

# -------------------------------
# 2. Tokenizer
# -------------------------------
business_category_token = Tokenizer() \
    .setInputCols(["business_category"]) \
    .setOutputCol("business_category_token")

# -------------------------------
# 3. Spell Checker
# -------------------------------
business_category_spell_checker = NorvigSweetingModel.pretrained() \
    .setInputCols(["business_category_token"]) \
    .setOutputCol("business_category_corrected")

# -------------------------------
# 4. Normalizer (lowercasing, clean text)
# -------------------------------
business_category_normalizer = Normalizer() \
    .setInputCols(["business_category_corrected"]) \
    .setOutputCol("business_category_normalized") \
    .setLowercase(True)

# -------------------------------
# 5. StopWords Cleaner
# -------------------------------
business_category_stopwordsCleaner = StopWordsCleaner() \
    .setInputCols(["business_category_normalized"]) \
    .setOutputCol("business_category_cleaned")

# -------------------------------
# 6. Lemmatizer
# -------------------------------
business_category_lemma = LemmatizerModel.pretrained() \
    .setInputCols(["business_category_token"]) \
    .setOutputCol("business_category_lemma")

# -------------------------------
# 7. Word Embeddings (GloVe)
# -------------------------------
business_category_glove_embeddings = WordEmbeddingsModel.pretrained("glove_100d") \
    .setInputCols(["business_category_token", "business_category"]) \
    .setOutputCol("word_embeddings")

# -------------------------------
# 8. Sentence Embeddings (average pooling)
# -------------------------------
business_category_sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["business_category", "word_embeddings"]) \
    .setOutputCol("business_category_embeddings") \
    .setPoolingStrategy("AVERAGE")

# -------------------------------
# 9. Embeddings Finisher (convert to Spark vector/array)
# -------------------------------
business_category_finisher = EmbeddingsFinisher() \
    .setInputCols(["business_category_embeddings"]) \
    .setOutputCols(["business_category_vector"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

In [16]:
# Combine both into one pipeline
pipeline = Pipeline(stages=[
    # --- Review branch ---
    customer_review,
    customer_review_token,
    customer_review_spell_checker,
    customer_review_normalizer,
    customer_review_stopwordsCleaner,
    customer_review_lemma,
    glove_embeddings,
    sentence_embeddings,
    customer_review_finisher,

    # --- Business category branch ---
    business_category,
    business_category_token,
    business_category_spell_checker,
    business_category_normalizer,
    business_category_stopwordsCleaner,
    business_category_lemma,
    business_category_glove_embeddings,
    business_category_sentence_embeddings,
    business_category_finisher
])


In [ ]:
result = pipeline.fit(df_joined).transform(df_joined)

In [18]:
result.select("text", "customer_review_vector").show(5, truncate=50, vertical=True)

-RECORD 0--------------------------------------------------------------------
 text                   | Lake filled with fish, actually a little hilly,... 
 customer_review_vector | [[-0.10794750601053238,0.2577876150608063,0.312... 
-RECORD 1--------------------------------------------------------------------
 text                   | Hot and fresh 🤩                                   
 customer_review_vector | [[-0.5256632566452026,0.3383975028991699,0.0488... 
-RECORD 2--------------------------------------------------------------------
 text                   | Always a long line, always busy                    
 customer_review_vector | [[-0.21003000438213348,0.2001621276140213,0.118... 
-RECORD 3--------------------------------------------------------------------
 text                   | Love the food                                      
 customer_review_vector | [[0.018898671492934227,0.41114330291748047,0.51... 
-RECORD 4--------------------------------------------------------

### Cosine similarity between review and business category

In [19]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
import numpy as np

def cosine_similarity(v1, v2):
    if v1 is None or v2 is None:
        return None
    a = np.asarray(v1, dtype=float)
    b = np.asarray(v2, dtype=float)

    # squeeze 1xN / Nx1 or nested singletons to 1-D
    if a.ndim > 1:
        a = a.reshape(-1)
    if b.ndim > 1:
        b = b.reshape(-1)

    na = np.linalg.norm(a)
    nb = np.linalg.norm(b)
    if na == 0 or nb == 0:
        return 0.0
    return float(np.dot(a, b) / (na * nb))

cosine_sim_udf = udf(cosine_similarity, DoubleType())

result = result.withColumn(
    "cosine_similarity",
    cosine_sim_udf("customer_review_vector", "business_category_vector")
)


In [ ]:
result.select("business_category", "text", "cosine_similarity").show(5, truncate=50)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+------------------+
|                                                                                   business_category|                                                                                                text| cosine_similarity|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+------------------+
|[{document, 0, 122, Equipment rental agency, Lawn equipment rental service, Party equipment renta...|Told me on the phone the equipment would fit in my car and could be operated by one person.  I do...|0.7935632383400323|
|[{document, 0, 75, Latin American restaurant, Bar, Live music venue, Lounge, Mexican restaurant, ...|Excell

## Sentiment analysis

In [22]:
from pyspark.storagelevel import StorageLevel

# Make sure types are friendly
result = (result
          .withColumn("rating", F.col("rating").cast("double"))
          .withColumn("text_nn", F.coalesce(F.col("text"), F.lit(""))))  # avoid nulls for NLP


In [ ]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, ViveknSentimentModel
from pyspark.ml import Pipeline

document = DocumentAssembler().setInputCol("text_nn").setOutputCol("doc")
token    = Tokenizer().setInputCols(["doc"]).setOutputCol("tok")
sent     = ViveknSentimentModel.pretrained().setInputCols(["doc","tok"]).setOutputCol("sentiment")
finish   = Finisher().setInputCols(["sentiment"]).setOutputCols(["sentiment_label"]).setOutputAsArray(True)

sent_pl  = Pipeline(stages=[document, token, sent, finish])
sent_m   = sent_pl.fit(result)

scored = sent_m.transform(result)

# Map to {-1,0,+1}
scored = (scored
  .withColumn("sentiment_str", F.element_at("sentiment_label", 1))
  .withColumn("sentiment_num",
              F.when(F.col("sentiment_str")=="positive", 1.0)
               .when(F.col("sentiment_str")=="negative", -1.0)
               .otherwise(0.0))
)


### Compare sentiment and vs star rating (for mismatches)

In [24]:
# 1..5 → [-1,+1]
scored = scored.withColumn("rating_norm", (F.col("rating") - 3.0) / 2.0)

# Text vs star disagreement
scored = scored.withColumn("sentiment_rating_gap", F.col("sentiment_num") - F.col("rating_norm"))
scored = scored.withColumn("mismatch_flag", F.abs(F.col("sentiment_rating_gap")) >= F.lit(0.8))  # tune


### “Non-visitor rant” heuristic (combine clues)

In [25]:
nonvisit_rx = r"""(?i)\b(
    never\s+been|haven'?t\s+been|didn'?t\s+visit|did\s+not\s+visit|
    phone\s+call|called\s+(them|store)|left\s+voicemail|email(ed)?|
    website|online\s+(order|booking|application)|
    recruiter|hiring|hr|application\s+process|
    delivery\s+app|uber\s+eats|doordash|grab\s+food
)\b"""

scored = (scored
  .withColumn("char_len", F.length("text_nn"))
  .withColumn("excl_count", F.size(F.split(F.regexp_replace(F.col("text_nn"), r"[^!]", ""), "")))
  .withColumn("caps_ratio",
              F.length(F.regexp_replace(F.col("text_nn"), r"[a-z]", "")) /
              (F.col("char_len") + F.lit(1)))
  .withColumn("nonvisit_clues", F.col("text_nn").rlike(nonvisit_rx))
)

# Strong negative + low topicality (cosine) + clues or shouty/short
scored = scored.withColumn(
    "possible_nonvisitor_rant",
    (F.col("sentiment_num") <= -0.5) &
    (F.col("cosine_similarity") < 0.25) &
    (F.col("nonvisit_clues") | (F.col("char_len") < 40) | (F.col("excl_count") >= 3))
)


### Low-quality / spam / off-topic flags

In [26]:
spam_rx = r"(?i)\b(whatsapp|http[s]?://|www\.|promo|discount code|coupon|call\s+\d{3,}|\+?\d{7,})\b"
irrelevant_rx = r"(?i)\b(politics|election|government|visa|immigration|job application|recruiter|hiring)\b"

scored = (scored
  .withColumn("has_link_or_phone", F.col("text_nn").rlike(spam_rx))
  .withColumn("irrelevant_general", F.col("text_nn").rlike(irrelevant_rx))
  .withColumn("low_quality_flag", F.col("has_link_or_phone") | F.col("irrelevant_general"))
)


### One score to triage

In [27]:
scored = scored.withColumn(
    "flag_score",
    (F.when(F.col("possible_nonvisitor_rant"), 3).otherwise(0)) +
    (F.when(F.col("low_quality_flag"),        2).otherwise(0)) +
    (F.when(F.col("mismatch_flag"),           1).otherwise(0)) +
    (F.when(F.col("cosine_similarity") < 0.15,1).otherwise(0))
)


In [28]:
keep_cols = ["gmap_id","business_name","category_str","rating","rating_norm",
             "sentiment_str","sentiment_num","cosine_similarity",
             "sentiment_rating_gap","mismatch_flag",
             "nonvisit_clues","low_quality_flag","possible_nonvisitor_rant",
             "flag_score","text_nn"]

final = scored.select(*keep_cols).persist(StorageLevel.MEMORY_AND_DISK)

In [ ]:
final.count()

In [ ]:
# A) Top suspicious
final.orderBy(F.desc("flag_score")).show(20, truncate=120)

In [ ]:
# B) Clear mismatches
final.filter(F.col("mismatch_flag")).select("rating","sentiment_str","sentiment_rating_gap","text_nn") \
     .show(10, truncate=120)

In [ ]:
# C) Category rollups
final.groupBy("category_str").agg(
    F.count("*").alias("n"),
    F.avg("rating").alias("avg_rating"),
    F.avg("sentiment_num").alias("avg_sentiment"),
    F.avg("cosine_similarity").alias("avg_cosine")
).orderBy(F.desc("n")).show(20, truncate=80)
